In [1]:
import numpy as np
from scipy.sparse import coo_matrix, vstack
from tqdm.notebook import tqdm

In [2]:
# check node update function
def MP_BEC_CN(x):
    y = np.full_like(x, 2) # all messages erased
    known = x != 2 # determine positions that are known (either 0 or 1)

    # Case 1, no erasure
    if np.all(known): 
        y[:] = np.prod(x)
    # Case 2, one single erasures
    elif np.sum(~known) == 1:
        y[~known] = np.prod(x[known])

    return y

# variable node 
def MP_BEC_VN(x, yc):
    y = np.full_like(x, yc)
    known = x != 2

    if yc == 2: # channel is erased
        if np.sum(known) == 1: # all erased but one message
            y[~known] = x[known][0]
        elif np.sum(known) > 1: # at least two non-erased messages
            y[:] = x[known][0]

    return y

# LDPC decoder for the BEC, inner loop
# completely vectorized but still relatively slow
# based on a sparse matrix as input
def decode_LDPC_BEC(y, H, iterations):
    m, n = H.shape # get row and column indices

    # initialize variable to check node messages with channel output
    VtoC = np.tile(y, (m, 1)) * H
    # initialize check to variable node messages with erasures
    CtoV = np.full((m, n), 2) * H

    # main iterations 
    for _ in range(iterations):

        # iterate through all check nodes  
        for i in range(m):
            idx = H[i] == 1
            CtoV[i, idx] = MP_BEC_CN(VtoC[i, idx])

        # iterate through all variable nodes
        for j in range(n):
            idx = H[:, j] == 1
            VtoC[idx, j] = MP_BEC_VN(CtoV[idx, j], y[j])

        # binary decision
        x_dec = y.copy()
        for j in range(n):
            msgs = CtoV[H[:, j] == 1, j]
            if np.any(msgs != 2):
                x_dec[j] = np.min(msgs)

        if np.all(x_dec < 2):
            xh = (1 - x_dec) // 2
            if np.all((H @ xh) % 2 == 0):
                # all parity-checks fulfilled?
                return xh

    return np.array([])

# generate a parity-check matrix according to Gallager's method
# do not care about 4-cycles
def generate_Gallager(dv, dc, n):
    if n % dc != 0:
        assert False, "n must be a multiple of check node degree dc"

    rows = n // dc
    # column indices
    jj = np.arange(n)
    ii = np.repeat(np.arange(rows), dc)
    Ho = coo_matrix((np.ones_like(jj), (ii, jj)), shape=(rows,n)).tocsr()
    H = Ho.copy()
    for _ in range(dv-1):
        H = vstack([H, Ho[:, np.random.permutation(n)]])
    
    return H

In [3]:
def simulate_LDPC_BEC_special():
    # parameters of regular LDPC code
    dv = 3
    dc = 6

    # specify epsilon (erasure probability) at which simulation takes place
    epsilon = 0.3

    # number of frames to simulate
    frames = 100

    # decoding iterations
    iterations = 50

    # generate parity-check matrix of regular LDPC code
    H = generate_Gallager(dv, dc, 240)

    n = H.shape[1]

    # simulate all-zero codeword
    x = np.zeros(n)
    
    errors = 0
    for frame in tqdm(range(frames)):
        print(f"{frame}/{frames}", end="\r")
        # erasure channel, first map to bipolar
        y = np.zeros(n, dtype=int)
        y[np.random.rand(n) < epsilon] = 2 # 2 denotes erasure

        xh = decode_LDPC_BEC(y, H.toarray(), iterations)

        errors = errors + (xh.size == 0)
    
    FER = errors / frames    # divide by two, as we may correctly guess the residual erasures
    print(f"epsilon = {epsilon:.2f}: FER = {FER:.4g}")

In [4]:
simulate_LDPC_BEC_special()

  0%|          | 0/100 [00:00<?, ?it/s]

epsilon = 0.30: FER = 0
